<a href="https://colab.research.google.com/github/HvTung04/NLP-UET-course_project/blob/main/Semantic%20similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

import numpy as np
import pandas as pd
import tensorflow as tf
import transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.8 MB/s eta 0:00:00


In [ ]:
max_length = 128  # Maximum length of input sentence to the model.
batch_size = 32
epochs = 2

# Labels in our dataset.
labels = ["similar", "not_similar"]

In [ ]:
# Read data in
train_df = pd.read_csv("train.tsv", sep='\t', on_bad_lines='skip')
test_df = pd.read_csv("test.tsv", sep='\t', on_bad_lines='skip')
dev_df = pd.read_csv("dev.tsv", sep='\t', on_bad_lines='skip')

In [ ]:
# Shape of the data
print(f"Total train samples: {train_df.shape[0]}")
print(f"Total test samples: {test_df.shape[0]}")
print(f"Total dev samples: {dev_df.shape[0]}")

Total train samples: 3458
Total test samples: 1639
Total dev samples: 480


In [ ]:
train_df.head(5)

,Quality,#1 ID,#2 ID,#1 String,#2 String
0,1,702876,702977,"Amrozi accused his brother, whom he called ""th...","Referring to him as only ""the witness"", Amrozi..."
1,0,2108705,2108831,Yucaipa owned Dominick's before selling the ch...,Yucaipa bought Dominick's in 1995 for $693 mil...
2,1,1330381,1330521,They had published an advertisement on the Int...,"On June 10, the ship's owners had published an..."
3,0,3344667,3344648,"Around 0335 GMT, Tab shares were up 19 cents, ...","Tab shares jumped 20 cents, or 4.6%, to set a ..."
4,1,1236820,1236712,"The stock rose $2.11, or about 11 percent, to ...",PG&E Corp. shares jumped $1.63 or 8 percent to...


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3458 entries, 0 to 3457
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Quality    3458 non-null   int64 
 1   #1 ID      3458 non-null   int64 
 2   #2 ID      3458 non-null   int64 
 3   #1 String  3458 non-null   object
 4   #2 String  3441 non-null   object
dtypes: int64(3), object(2)
memory usage: 135.2+ KB


In [ ]:
# Delete null objects
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)
dev_df.dropna(inplace=True)

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3441 entries, 0 to 3457
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Quality    3441 non-null   int64 
 1   #1 ID      3441 non-null   int64 
 2   #2 ID      3441 non-null   int64 
 3   #1 String  3441 non-null   object
 4   #2 String  3441 non-null   object
dtypes: int64(3), object(2)
memory usage: 161.3+ KB


In [ ]:
# Train target distribution
print("Train target distribution")
print(train_df.Quality.value_counts())

Train target distribution
1    2316
0    1125
Name: Quality, dtype: int64


In [ ]:
# Valid target distribution
print("Valid target distribution")
print(dev_df.Quality.value_counts())

Valid target distribution
1    330
0    146
Name: Quality, dtype: int64


In [ ]:
train_df["similarity"] = train_df["Quality"].apply(
    lambda x: "not_similar" if x == 0 else "similar"
)
y_train = tf.keras.utils.to_categorical(train_df.Quality, num_classes=2)

dev_df["similarity"] = dev_df["Quality"].apply(
    lambda x: "not_similar" if x == 0 else "similar"
)
y_dev = tf.keras.utils.to_categorical(dev_df.Quality, num_classes=2)

test_df["similarity"] = test_df["Quality"].apply(
    lambda x: "not_similar" if x == 0 else "similar"
)
y_test = tf.keras.utils.to_categorical(test_df.Quality, num_classes=2)

In [ ]:
class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data.

    Args:
        sentence_pairs: Array of premise and hypothesis input sentences.
        labels: Array of labels.
        batch_size: Integer batch size.
        shuffle: boolean, whether to shuffle the data.
        include_targets: boolean, whether to include the labels.

    Returns:
        Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
        (or just `[input_ids, attention_mask, `token_type_ids]`
         if `include_targets=False`)
    """

    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load our BERT Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "bert-base-uncased", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return (len(self.sentence_pairs) + self.batch_size - 1) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        start = idx * self.batch_size
        end = (idx + 1) * self.batch_size

    # Make sure 'end' does not go beyond the end of the dataset.
        if end > len(self.sentence_pairs):
          end = len(self.sentence_pairs)

        indexes = self.indexes[start:end]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
            truncation = True,
            padding = 'max_length'
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)


In [ ]:
# Create the model under a distribution strategy scope.
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Encoded token ids from BERT tokenizer.
    input_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="input_ids"
    )
    # Attention masks indicates to the model which tokens should be attended to.
    attention_masks = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="attention_masks"
    )
    # Token type ids are binary masks identifying different sequences in the model.
    token_type_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="token_type_ids"
    )
    # Loading pretrained BERT model.
    bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased")
    # Freeze the BERT model to reuse the pretrained features without modifying them.
    bert_model.trainable = False

    bert_output = bert_model.bert(
        input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
    )
    sequence_output = bert_output.last_hidden_state
    pooled_output = bert_output.pooler_output
    # Add trainable layers on top of frozen layers to adapt the pretrained features on the new data.
    bi_lstm = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64, return_sequences=True)
    )(sequence_output)
    # Applying hybrid pooling approach to bi_lstm sequence output.
    avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_lstm)
    max_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_lstm)
    concat = tf.keras.layers.concatenate([avg_pool, max_pool])
    dropout = tf.keras.layers.Dropout(0.3)(concat)
    output = tf.keras.layers.Dense(2, activation="softmax")(dropout)
    model = tf.keras.models.Model(
        inputs=[input_ids, attention_masks, token_type_ids], outputs=output
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="categorical_crossentropy",
        metrics=["acc"],
    )


print(f"Strategy: {strategy}")
model.summary()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Strategy: <tensorflow.python.distribute.mirrored_strategy.MirroredStrategy object at 0x7e6c4c0377c0>
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 128)]                0         []                            
                                                                                                  
 attention_masks (InputLaye  [(None, 128)]                0         []                            
 r)                                                                                               
                                                                                                  
 token_type_ids (InputLayer  [(None, 128)]                0         []                            
 )                                                                                          

In [ ]:
train_data = BertSemanticDataGenerator(
    train_df[["#1 String", "#2 String"]].values.astype("str"),
    y_train,
    batch_size=batch_size,
    shuffle=True,
)
valid_data = BertSemanticDataGenerator(
    dev_df[["#1 String", "#2 String"]].values.astype("str"),
    y_dev,
    batch_size=batch_size,
    shuffle=True,
)

In [ ]:
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs= 2,
    validation_steps = len(valid_data) // batch_size,
    use_multiprocessing=True,
    workers=-1,
)

Epoch 1/2
 23/108 [=====>........................] - ETA: 24:05 - loss: 0.6435 - acc: 0.6332

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


 31/108 [=======>......................] - ETA: 21:42 - loss: 0.6236 - acc: 0.6583

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


 77/108 [====================>.........] - ETA: 8:46 - loss: 0.6131 - acc: 0.6753

...

 66/108 [=================>............] - ETA: 12:10 - loss: 0.5461 - acc: 0.7202

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


 84/108 [======================>.......] - ETA: 6:55 - loss: 0.5440 - acc: 0.7206

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


108/108 [==============================] - 1853s 17s/step - loss: 0.5403 - acc: 0.7225


In [ ]:
# Unfreeze the bert_model.
bert_model.trainable = True
# Recompile the model to make the change effective.
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 128)]                0         []                            
                                                                                                  
 attention_masks (InputLaye  [(None, 128)]                0         []                            
 r)                                                                                               
                                                                                                  
 token_type_ids (InputLayer  [(None, 128)]                0         []                            
 )                                                                                                
                                                                                              

In [ ]:
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
)

Epoch 1/2


 51/108 [=============>................] - ETA: 46:18 - loss: 0.4790 - accuracy: 0.7594

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


 61/108 [===============>..............] - ETA: 38:20 - loss: 0.4769 - accuracy: 0.7646

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


...

 60/108 [===============>..............] - ETA: 38:52 - loss: 0.3946 - accuracy: 0.8257

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


108/108 [==============================] - ETA: 0s - loss: 0.3867 - accuracy: 0.8245 

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


108/108 [==============================] - 5532s 51s/step - loss: 0.3867 - accuracy: 0.8245 - val_loss: 0.4314 - val_accuracy: 0.7962


In [ ]:
def check_similarity(sentence1, sentence2):
    sentence_pairs = np.array([[str(sentence1), str(sentence2)]])
    test_data = BertSemanticDataGenerator(
        sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
    )

    proba = model.predict(test_data[0])[0]
    idx = np.argmax(proba)
    proba = f"{proba[idx]: .2f}"
    pred = labels[idx]
    return pred, proba


In [ ]:
sentence1 = "Today is very hot."
sentence2 = "It is freezing outside today."
check_similarity(sentence1, sentence2)

1/1 [==============================] - 1s 985ms/step


('not_similar', ' 0.60')

In [ ]:
sentence1 = "A soccer game with multiple males playing"
sentence2 = "Some men are playing sport"
check_similarity(sentence1, sentence2)

1/1 [==============================] - 1s 605ms/step


('similar', ' 0.72')

In [ ]:
sentence1 = "Amrozi accused his brother, whom he called the witness, of deliberately distorting his evidence."
sentence2 = "Referring to him as only the witness, Amrozi accused his brother of deliberately distorting his evidence."
check_similarity(sentence1, sentence2)

1/1 [==============================] - 1s 609ms/step


('similar', ' 0.94')